In [55]:
import nltk
nltk.download('nps_chat')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\emiel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
posts = nltk.corpus.nps_chat.xml_posts()
X = [post.text for post in posts]
y  = ['question' if (post.get('class') == 'whQuestion' or post.get('class') == 'ynQuestion') else "other" for post in posts]
translate_dict = {'question':1, 'other':0}

y = [translate_dict[entry] for entry in y]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

# NLTK Naive Bayes Classifier

++ easy to understand

-- ignores word order / context ("is this it" is the same as "this is it") confuses statements and questions

-- huge reliance on signal words

In [3]:

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features


featuresets = [(dialogue_act_features(post.text), target) for post, target in list(zip(posts,y))]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.8731060606060606


In [4]:
set([post.get('class') for post in posts])

{'Accept',
 'Bye',
 'Clarify',
 'Continuer',
 'Emotion',
 'Emphasis',
 'Greet',
 'Other',
 'Reject',
 'Statement',
 'System',
 'nAnswer',
 'whQuestion',
 'yAnswer',
 'ynQuestion'}

In [5]:
classifier.show_most_informative_features(50)

Most Informative Features
             contains(?) = True                1 : 0      =     59.5 : 1.0
           contains(wtf) = True                1 : 0      =     56.8 : 1.0
         contains(whats) = True                1 : 0      =     54.6 : 1.0
       contains(anybody) = True                1 : 0      =     44.8 : 1.0
           contains(asl) = True                1 : 0      =     44.8 : 1.0
            contains(tx) = True                1 : 0      =     32.9 : 1.0
         contains(which) = True                1 : 0      =     32.9 : 1.0
           contains(who) = True                1 : 0      =     31.1 : 1.0
          contains(part) = True                0 : 1      =     30.0 : 1.0
          contains(doin) = True                1 : 0      =     26.9 : 1.0
        contains(purple) = True                1 : 0      =     26.9 : 1.0
           contains(how) = True                1 : 0      =     25.0 : 1.0
        contains(anyone) = True                1 : 0      =     23.4 : 1.0

In [6]:
line = "Do you enjoy high performance programming?"
print(classifier.classify(dialogue_act_features(line)))


1


In [7]:
line = "What do you think about high performance programming?"
print(classifier.classify(dialogue_act_features(line)))

1


In [8]:
line = "i am testing this"
print(classifier.classify(dialogue_act_features(line)))

0


# Small Bert
https://www.tensorflow.org/text/tutorials/classify_text_with_bert

In [9]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [10]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [11]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [12]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [13]:
bert_model(text_preprocessed)

{'sequence_output': <tf.Tensor: shape=(1, 128, 512), dtype=float32, numpy=
 array([[[-0.2894631 ,  0.3432125 ,  0.33231503, ...,  0.21300879,
           0.71020824, -0.05771083],
         [-0.28741956,  0.31981033, -0.23018478, ...,  0.58455044,
          -0.21329702,  0.72692096],
         [-0.66156983,  0.68876874, -0.87432986, ...,  0.10877332,
          -0.26173213,  0.4785546 ],
         ...,
         [ 0.07634938, -0.14029206, -0.23991814, ...,  0.43109894,
           0.9132698 ,  0.2148659 ],
         [-0.44950542, -0.4125196 , -0.18450823, ...,  0.42683274,
           0.57654595,  0.5835239 ],
         [-1.012537  , -0.8638922 ,  0.05829375, ...,  0.33312175,
           0.36520925,  0.94291836]]], dtype=float32)>,
 'default': <tf.Tensor: shape=(1, 512), dtype=float32, numpy=
 array([[ 7.62628734e-01,  9.92809832e-01, -1.86118647e-01,
          3.66738200e-01,  1.52337536e-01,  6.55044615e-01,
          9.68115389e-01, -9.48626995e-01,  2.16128677e-03,
         -9.87773120e-01, 

In [40]:
def build_classifier_model():
    
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [41]:
classifier_model = build_classifier_model()

In [42]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [43]:
epochs = 5
init_lr = 3e-5
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [5e-5, 3e-5, 2e-5])
wd = lambda: 1e-4 * schedule(step)

optimizer = tfa.optimizers.AdamW(learning_rate=init_lr, weight_decay=wd)


In [44]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [45]:
history = classifier_model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=epochs)

Epoch 1/5
265/265 [==============================] - 71s 253ms/step - loss: 0.0993 - binary_accuracy: 0.9656 - val_loss: 0.0707 - val_binary_accuracy: 0.9792
Epoch 2/5
265/265 [==============================] - 68s 256ms/step - loss: 0.0541 - binary_accuracy: 0.9820 - val_loss: 0.0673 - val_binary_accuracy: 0.9797
Epoch 3/5
265/265 [==============================] - 67s 253ms/step - loss: 0.0362 - binary_accuracy: 0.9885 - val_loss: 0.0698 - val_binary_accuracy: 0.9801
Epoch 4/5
265/265 [==============================] - 67s 251ms/step - loss: 0.0221 - binary_accuracy: 0.9923 - val_loss: 0.0874 - val_binary_accuracy: 0.9816
Epoch 5/5
265/265 [==============================] - 67s 252ms/step - loss: 0.0208 - binary_accuracy: 0.9936 - val_loss: 0.0828 - val_binary_accuracy: 0.9740


In [ ]:
preds = classifier_model.predict(X_test)

In [48]:
def conv_neg_pos(input):
    if input < 0: return 0
    else: return 1
    

In [51]:
preds = [conv_neg_pos(x) for x in preds]

In [57]:
accuracy_score(y_test, preds)

0.9843897824030274

In [63]:
question_indices = list(filter(None,[i  if preds[i] == 1 else None for i in range(len(preds))]))

In [67]:
questions = [X_test[q] for q in question_indices]

In [68]:
questions

["does anyone want to talk to me on the phone?  I don't charge",
 'coexistence?',
 'who, me?',
 "who's a virgin?",
 'whats everyone up to?',
 "10-19-adultsUser15: What's your sign?",
 'what 10-19-30sUser11',
 'if he cant get a pro 11-09-40sUser7, why bother',
 'whats going on 11-08-teensUser22...',
 "hey! you aren't making fun of me are you?",
 'did i lose 11-06-adultsUser19?',
 'So?',
 'what happened to the girl w/ the same birthday as me??????',
 'elbows? its not knees anymore??',
 'thats why i dont cyber. whats the point unless you cant get any in person?',
 'up in this biyatch? :)',
 'anybody here know how to play heartbreaker on the electric guitar?',
 'why 10-19-adultsUser35...there r plently  ladies in here with pink pussies',
 'what an invention',
 'any guys wanna chat with me',
 'lookin good fella?',
 'u need paper twoel?',
 'why?',
 'awww, 10-24-40sUser41 dont be mad?',
 'you mean your cooter isnt leakin fruit punch?',
 'huh?',
 'what you do for fun',
 'are you kiddin me',
 '